# Google Generative AI and CSV Processing

## Installation

First, ensure that you have the required package installed. You can do this by running:

```python
%pip install -U -q "google-generativeai>=0.8.3"

In [1]:
%pip install -U -q "google-generativeai>=0.8.3"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

## Configuration
Set your Google API key for the generative AI model:


In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## File Descriptions

- **`Final_evaluated_1.csv`**
  - **Description**: This file contains the results for **Model Version 1** after human evaluation.
  - **Content**: The output includes performance metrics and evaluation results obtained after human review.

- **`Final_evaluated_2.csv`**
  - **Description**: This file contains the results for **Model Version 2** after human evaluation.
  - **Content**: Similar to the first file, it includes performance metrics and evaluation results after being assessed by human evaluators.

- **`Final_evaluated_3.csv`**
  - **Description**: This file contains the results for **Model Version 3** after human evaluation.
  - **Content**: This file also includes performance metrics and evaluation results from human review.

In [4]:
import pandas as pd
import re 

# Parameters
file_paths = [
    '/kaggle/input/model-1data/model_responses_with_time_tqdm1S .csv',
    '/kaggle/input/model-2data/model_responses_with_time_tqdm2S.csv',
    '/kaggle/input/model-3data/model_responses_with_time_tqdm3S.csv'
]

output_paths = [
    './Final_evaluated_1.csv',
    './Final_evaluated_2.csv',
    './Final_evaluated_3.csv'
]

max_tokens = 200  # Maximum tokens for the model response

# Initialize the Gemini model for evaluating answers.
short_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(max_output_tokens=max_tokens)
)

def extract_and_evaluate(row):
    """
    Evaluates the model's answer against the expected answer.

    Parameters:
    row (pd.Series): A row of the DataFrame containing 'question', 'expected_answer', 'model_answer', and 'Max_Score'.

    Returns:
    tuple: Extracted model answer, score, and reasoning.
    """
    question = row['question']
    expected_answer = row['expected_answer']
    model_answer = row['model_answer']
    max_score = row['Max_Score']

    # Extract the part of the model answer that is actually the answer
    answer_start = model_answer.find("Answer:") + len("Answer:")
    model_answer_extracted = model_answer[answer_start:].strip() if answer_start > -1 else "No valid answer found."

    # Prepare the evaluation prompt
    prompt = (
        f"Given the question below, evaluate the provided answer and score it from 0 to {max_score} based on its alignment with the expected answer.\n"
        f"Question: {question}\n"
        f"Expected Answer: {expected_answer}\n"
        f"Model Answer: {model_answer_extracted}\n"
        "Please provide a score and brief reasoning, focusing on conceptual alignment."
    )

    # Use the model to evaluate the extracted answer
    try:
        evaluation_response = short_model.generate_content(prompt).text.strip()
        
        # Extract score and reasoning using regex
        score_match = re.search(r'Score:\s*[\*]*(\d*\.?\d+)[\*]*', evaluation_response)
        score = float(score_match.group(1)) if score_match else 0
        
        reasoning_match = re.search(r'Reasoning:\s*([\s\S]*)', evaluation_response)
        reasoning = reasoning_match.group(1).strip() if reasoning_match else "Reasoning not provided."
        
    except Exception as e:
        print(f"Error parsing evaluation response for question '{question}': {e}")
        score = 0
        reasoning = "Error in evaluation."

    return model_answer_extracted, score, reasoning

# Process each file
for i, (data_path, output_path) in enumerate(zip(file_paths, output_paths)):
    # Load the dataset
    df = pd.read_csv(data_path)  # Make sure this path is correct

    # Apply the function to all rows in the DataFrame
    results = df.apply(extract_and_evaluate, axis=1)

    # Expand the results into separate columns
    df['Extracted Model Answer'] = results.str[0]
    df['Gemini Score'] = results.str[1]
    df['Evaluation Reasoning'] = results.str[2]

    # Save the updated DataFrame back to CSV
    df.to_csv(output_path, index=False)

    # Print the sum of the Gemini Score for the current file
    total_score = df['Gemini Score'].sum()
    print(f"Total Gemini Score for {data_path}: {total_score}")

Total Gemini Score for /kaggle/input/model-1data/model_responses_with_time_tqdm1S .csv: 45.9
Total Gemini Score for /kaggle/input/model-2data/model_responses_with_time_tqdm2S.csv: 48.58
Total Gemini Score for /kaggle/input/model-3data/model_responses_with_time_tqdm3S.csv: 47.3
